In [2]:


output_dir = '../performance'


import pandas as pd
import os
import json

output_dir = '../performance'

queries = {
    1: '(Graph structured) AND (visual Question Answering)',
    2: '"Autonomous Surface Vehicles" AND "Federated Learning"',
    3: 'RAG AND (Long-context LLM) OR (long context language model)',
    4: 'Applications of visual question answering systems',
    5: 'Fairness AND socials',
    6: '(Regulatory Capture AND IA OR ML) OR (Regulatory Capture OR Machine Learning)',
    7: '(Segmentation OR SAM) AND failure',
    8: '(Graph Convolutional Network) OR (GCN) AND ',
    9: '((Question Answering) AND (visual)) OR (VQA)',
    10: 'FL AND decentralised',
}

In [15]:


results = []
for file in os.listdir(output_dir):
    with open(f'{output_dir}/{file}', 'r') as f:
        response = f.read()
        
        # remove all that is not json
        response = response.split('{')[1].split('}')[0]
        response = '{' + response + '}'
                
        try:
            response = json.loads(response)
        except json.JSONDecodeError as e:
            print(e)
            print(f'Error in file {file}')
            continue

        score = response.get('score', None)
        relevant = response.get('relevant', None)
        explanation = response.get('explanation', None)
        keywords = response.get('keywords', None)
        
       
        index, query = file.split('_')
        query = query.split('.')[0]
  

        results.append({
            'index': index,
            'query_id': query,
            'score': score,
            'relevant': relevant,
            'explanation': explanation,
            'keywords': keywords
        })

df_results = pd.DataFrame(results)
df_results.to_csv('llm_response.csv', index=False)


In [4]:
data = pd.read_csv('../gym/data.csv')
llm_res = pd.read_csv('../gym/llm_response.csv')
altair_res = pd.read_csv('../gym/altair_response.csv')

#join llm_res index with data on id

llm_res = llm_res.merge(data, left_on='index', right_on='index')
llm_res = llm_res[['query_id', 'index', 'score', 'relevant','file_id', 'table_id']]

llm_res['id'] = llm_res['file_id'].astype(str) + '_' + llm_res['table_id'].astype(str)
altair_res['id'] = altair_res['file_id'].astype(str) + '_' + altair_res['table_id'].astype(str)

#export to csv
llm_res.to_csv('llm_response.csv', index=False)
altair_res.to_csv('altair_response.csv', index=False)